In [1]:
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout


In [2]:
import pandas as pd
file_path = 'wheather_data.csv'
reviews = pd.read_csv(file_path) 
#pd.set_option("display.max_rows", 8)
dataset = reviews.values
print(reviews)

             id            timestamp    value identifier  value_type_id  \
0        427436  2019-04-25 13:20:09  1021.80       PMSL             20   
1        427439  2019-04-25 13:20:09    12.00        TDP             15   
2        427434  2019-04-25 13:20:09     0.00        P6H             13   
3        427435  2019-04-25 13:20:09     0.51       P24H             14   
4        427440  2019-04-25 13:20:09     3.00        UVI             16   
...         ...                  ...      ...        ...            ...   
406714  6426613  2019-07-29 23:40:08    23.00          T             11   
406715  6426610  2019-07-29 23:40:08     0.00       P24H             14   
406716  6426611  2019-07-29 23:40:08  1007.10       PMSL             55   
406717  6426617  2019-07-29 23:40:08     0.00         WS             18   
406718  6426596  2019-07-29 23:40:08   310.00         WD             17   

        location_id  source_id  
0              23.0          5  
1              23.0          5  


In [3]:
reviews['timestamp'] = pd.to_datetime(reviews['timestamp'], format = '%Y-%m-%d %H:%M:%S')#.dt.datetime
reviews['month'] = reviews.timestamp.dt.month
reviews['day'] = reviews.timestamp.dt.day
reviews['date'] = reviews.timestamp.dt.date
reviews['unix'] = pd.DatetimeIndex(reviews.date).astype(np.int32)
reviews = reviews.drop('timestamp', axis = 1)
print(reviews)

             id    value identifier  value_type_id  location_id  source_id  \
0        427436  1021.80       PMSL             20         23.0          5   
1        427439    12.00        TDP             15         23.0          5   
2        427434     0.00        P6H             13         23.0          5   
3        427435     0.51       P24H             14         23.0          5   
4        427440     3.00        UVI             16         23.0          5   
...         ...      ...        ...            ...          ...        ...   
406714  6426613    23.00          T             11        116.0          5   
406715  6426610     0.00       P24H             14        116.0          5   
406716  6426611  1007.10       PMSL             55        116.0          5   
406717  6426617     0.00         WS             18        116.0          5   
406718  6426596   310.00         WD             17         30.0          5   

        month  day        date                 unix  
0        

<ipython-input-3-78eccf365fad>:5: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  reviews['unix'] = pd.DatetimeIndex(reviews.date).astype(np.int32)


In [17]:
y = reviews.value.loc[1:]
y = np.array(y)
y = y.reshape(-1,1)


df_features = ['value_type_id', 'value','month', 'day']
X = reviews[df_features]
X = X.iloc[:-1]
#X = X.values
print(y)
print(X)

[[1.2000e+01]
 [0.0000e+00]
 [5.1000e-01]
 ...
 [1.0071e+03]
 [0.0000e+00]
 [3.1000e+02]]
        value_type_id    value  month  day
0                  20  1021.80      4   25
1                  15    12.00      4   25
2                  13     0.00      4   25
3                  14     0.51      4   25
4                  16     3.00      4   25
...               ...      ...    ...  ...
406713             12     0.00      7   29
406714             11    23.00      7   29
406715             14     0.00      7   29
406716             55  1007.10      7   29
406717             18     0.00      7   29

[406718 rows x 4 columns]


In [18]:
scaler = MinMaxScaler()
X_scale= scaler.fit_transform(X) 
print(X_scale.shape[1])


4


In [19]:
y_scale= scaler.fit_transform(y.reshape(-1, 1))
print(y_scale)


[[1.17612061e-02]
 [1.85924716e-05]
 [5.17653552e-04]
 ...
 [9.85517443e-01]
 [1.85924716e-05]
 [3.03369445e-01]]


In [20]:
X_scale = np.reshape(X_scale, (int(X_scale.shape[0]/2) , 2 , X_scale.shape[1]))
y_scale = np.reshape(y_scale, (int(y_scale.shape[0]/2) , 2 , y_scale.shape[1]))

In [21]:
X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(X_scale, y_scale, test_size=0.3)
X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)

print(X_train.shape)
print(X_test.shape)

(142351, 2, 4)
(30504, 2, 4)


In [22]:
tf.keras.layers.InputLayer(input_shape = X_train.shape[2])
                           
rnn = Sequential()
                           
rnn.add(LSTM(units = 45, return_sequences = True))



In [23]:
for i in [True, True, False]:  #3 ltsm layers, false to teleutaio gia n mn mpei tetarto
    rnn.add(LSTM(units = 45, return_sequences = i))
    rnn.add(Dropout(0.2))   #dropping 20%
    
#return_sequences = True - this must always be specified if you plan on including another LSTM layer after the one you're adding. 
#You should specify return_sequences = False for the last LSTM layer in your recurrent neural network.   

In [24]:
rnn.add(Dense(units = 1)) #1 eksodo

In [25]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)

In [26]:
rnn.compile(optimizer = opt, loss = 'mean_squared_error')

In [27]:
rnn.fit(X_train, Y_train, epochs = 20, batch_size = 500)

Epoch 1/20
285/285 [==============================] - 13s 19ms/step - loss: 0.0877
Epoch 2/20
285/285 [==============================] - 6s 20ms/step - loss: 0.0779
Epoch 3/20
285/285 [==============================] - 6s 20ms/step - loss: 0.0776
Epoch 4/20
285/285 [==============================] - 5s 19ms/step - loss: 0.0775
Epoch 5/20
285/285 [==============================] - 6s 20ms/step - loss: 0.0774
Epoch 6/20
285/285 [==============================] - 6s 22ms/step - loss: 0.0773
Epoch 7/20
285/285 [==============================] - 5s 19ms/step - loss: 0.0773
Epoch 8/20
285/285 [==============================] - 5s 18ms/step - loss: 0.0773
Epoch 9/20
285/285 [==============================] - 6s 20ms/step - loss: 0.0773
Epoch 10/20
285/285 [==============================] - 5s 18ms/step - loss: 0.0772
Epoch 11/20
285/285 [==============================] - 6s 19ms/step - loss: 0.0772
Epoch 12/20
285/285 [==============================] - 5s 19ms/step - loss: 0.0772
Epoch 13/20


In [15]:
predictions = rnn.predict(X_test)
print(predictions)
#plt.plot(predictions)

[[0.14758424]
 [0.10719644]
 [0.08861003]
 ...
 [0.10006165]
 [0.08015412]
 [0.08589125]]


In [16]:
unscaled_predictions = scaler.inverse_transform(predictions)
print(unscaled_predictions)

[[ 68.7344  ]
 [ 81.70689 ]
 [ 81.45553 ]
 ...
 [ 76.0779  ]
 [ 45.092247]
 [116.20332 ]]


In [28]:
X = pd.DataFrame(X)

pd.DataFrame.corr(X,"spearman")

,value_type_id,value,month,day
value_type_id,1.000000,0.255192,0.126457,0.261311
value,0.255192,1.000000,0.012571,-0.027193
month,0.126457,0.012571,1.000000,-0.364762
day,0.261311,-0.027193,-0.364762,1.000000
